In [1]:
#Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas_ta as ta
import yfinance as yf
from backtesting import Backtest, Strategy
from alpha_vantage.timeseries import TimeSeries
import requests

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#OLD CODE USING YFINANCE
"""#Download asset data
df = yf.download(tickers="AAPL", start='2010-01-01', end='2023-12-31')

#calculatre technical indicators
df['MA'] = ta.ema(df['Close'], length=20)
df['rsi'] = ta.rsi(df['Close'], length=14)
df['coppock'] = ta.coppock(df['Close'], length=12)
df['returns'] = df['Close'].pct_change(-1) * 100 * -1
df['forecast_tommorrow'] = np.where(df['returns'] > 0,
                                   1,
                                   0) #1 is for UP and 0 is for down

df = df.dropna()
df"""

'#Download asset data\ndf = yf.download(tickers="AAPL", start=\'2010-01-01\', end=\'2023-12-31\')\n\n#calculatre technical indicators\ndf[\'MA\'] = ta.ema(df[\'Close\'], length=20)\ndf[\'rsi\'] = ta.rsi(df[\'Close\'], length=14)\ndf[\'coppock\'] = ta.coppock(df[\'Close\'], length=12)\ndf[\'returns\'] = df[\'Close\'].pct_change(-1) * 100 * -1\ndf[\'forecast_tommorrow\'] = np.where(df[\'returns\'] > 0,\n                                   1,\n                                   0) #1 is for UP and 0 is for down\n\ndf = df.dropna()\ndf'

In [3]:
def ticker_data_download(ticker):
    """
    Function to download ticker daily data from alpha vantage.
    It returns a pandas data frame.
    Use your own API.
    You need to import library pandas_ta
    """
    #import alpha vantage API from a text file
    with open('myAPI.txt', 'r') as file:
        api_key = file.read()
    # Initialize Alpha Vantage API client
    ts = TimeSeries(key=api_key, output_format='pandas')
    symbol = ticker
    #Download ticker data
    data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
    df = data
    #rename columns
    df.rename(columns={
        '1. open': 'Open',
        '2. high': 'High',
        '3. low': 'Low',
        '4. close': 'Close',
        '5. volume': 'Volume'
    }, inplace=True)
    #Calculate indicators with pandas_ta
    df['MA'] = ta.ema(df['Close'], length=20)
    df['rsi'] = ta.rsi(df['Close'], length=14)
    df['coppock'] = ta.coppock(df['Close'], length=12)
    df['returns'] = df['Close'].pct_change(-1) * 100 * -1
    df['forecast_tommorrow'] = np.where(df['returns'] > 0,
                                       1,
                                       0) #1 is for UP and 0 is for down
    df = df.dropna()
    return df

In [4]:
df = ticker_data_download('IBM')
df

,Open,High,Low,Close,Volume,MA,rsi,coppock,returns,forecast_tommorrow
date,,,,,,,,,,
2024-01-08,158.69,161.216,157.885,161.14,3321698.0,172.804217,23.029073,-22.422795,-1.244031,0
2024-01-05,159.91,160.550,158.670,159.16,3698961.0,171.504767,21.228994,-23.040880,1.056820,1
2024-01-04,160.22,161.810,160.170,160.86,3212004.0,170.490980,26.538377,-23.324857,-0.474703,0
2024-01-03,161.00,161.730,160.080,160.10,4086065.0,169.501363,25.704250,-22.263335,0.866873,1
2024-01-02,162.83,163.290,160.460,161.50,3825044.0,168.739328,30.064907,-20.530118,1.253439,1
...,...,...,...,...,...,...,...,...,...,...
1999-11-08,90.50,93.940,90.500,93.94,8522300.0,100.260386,34.731247,-21.713106,-4.088643,0
1999-11-05,92.75,92.940,90.190,90.25,13737600.0,99.307016,30.510807,-23.145898,1.430756,1
1999-11-04,94.44,94.440,90.000,91.56,16697600.0,98.569205,33.595856,-23.372568,2.977641,1


In [5]:
# Separate features and target variable
X = df.drop(columns=['forecast_tommorrow'])
y = df['forecast_tommorrow']

In [6]:
#split data into trains and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#scale variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
model = XGBClassifier()

In [9]:
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [10]:
y_pred = model.predict(X_test_scaled)
y_pred

array([1, 1, 1, ..., 1, 0, 1])

In [11]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9958915365653246
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       612
           1       1.00      0.99      1.00       605

    accuracy                           1.00      1217
   macro avg       1.00      1.00      1.00      1217
weighted avg       1.00      1.00      1.00      1217



In [12]:
compare = pd.DataFrame(y_test)
compare['predictions'] = y_pred
compare.head(50)

,forecast_tommorrow,predictions
date,,
2019-11-08,1,1
2007-11-21,1,1
2005-09-23,1,1
2014-06-11,1,1
2023-05-09,1,1
2016-06-29,0,0
2004-10-08,1,1
2002-04-04,0,0
2002-09-25,0,0


Aplicaçao do modelo em novos dados

In [13]:
#Download asset data
df_new_stock = yf.download(tickers="MSFT")

#calculatre technical indicators
df_new_stock['MA'] = ta.ema(df_new_stock['Close'], length=20)
df_new_stock['rsi'] = ta.rsi(df_new_stock['Close'], length=14)
df_new_stock['coppock'] = ta.coppock(df_new_stock['Close'], length=12)
df_new_stock['returns'] = df_new_stock['Close'].pct_change(-1) * 100 * -1
df_new_stock['forecast_tommorrow'] = np.where(df_new_stock['returns'] > 0,
                                   1,
                                   0) #1 is for UP and 0 is for down
df_new_stock = df_new_stock.drop(['Adj Close'], axis=1)
df_new_stock = df_new_stock.dropna()
df_new_stock

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,MA,rsi,coppock,returns,forecast_tommorrow
Date,,,,,,,,,,
1986-04-18 00:00:00-05:00,0.105035,0.105035,0.100694,0.101563,21628800,0.098896,59.005425,12.142096,-0.000000,0
1986-04-21 00:00:00-05:00,0.101563,0.102431,0.098958,0.101563,22924800,0.099150,59.005425,12.943036,-1.740026,0
1986-04-22 00:00:00-05:00,0.101563,0.101563,0.099826,0.099826,15552000,0.099214,53.351414,13.109644,0.432871,1
1986-04-23 00:00:00-05:00,0.099826,0.100694,0.098958,0.100260,15609600,0.099314,54.523930,12.916247,9.055453,1
1986-04-24 00:00:00-05:00,0.100260,0.111979,0.099826,0.110243,62352000,0.100355,71.974104,15.382587,5.926374,1
...,...,...,...,...,...,...,...,...,...,...
2024-02-06 00:00:00-05:00,405.880005,407.970001,402.910004,405.489990,18382600,397.776708,61.796883,10.250556,2.067383,1
2024-02-07 00:00:00-05:00,407.440002,414.299988,407.399994,414.049988,22340500,399.326544,67.535102,9.764800,0.014488,1
2024-02-08 00:00:00-05:00,414.049988,415.559998,412.529999,414.109985,21225300,400.734491,67.571868,8.998705,1.531329,1


In [14]:
# Separate features and target variable
X_new_stock = df_new_stock.drop(columns=['forecast_tommorrow'])
y_new_stock = df_new_stock['forecast_tommorrow']

In [15]:
scaler = StandardScaler()
X_new_stock_scaled = scaler.fit_transform(X_new_stock)
X_new_stock_scaled

array([[-0.6656393 , -0.66607467, -0.6653062 , ...,  0.44842923,
         0.80691063, -0.02997465],
       [-0.6656821 , -0.66610646, -0.66532782, ...,  0.44842923,
         0.87522726, -0.84157599],
       [-0.6656821 , -0.66611705, -0.66531701, ..., -0.03377989,
         0.8894382 ,  0.17192954],
       ...,
       [ 4.43728607,  4.4049873 ,  4.47116097, ...,  1.17902865,
         0.5387932 ,  0.68428379],
       [ 4.45207927,  4.46919122,  4.50304363, ...,  1.49916957,
         0.525513  , -0.62415834],
       [ 4.51753851,  4.46821453,  4.49880926, ...,  0.93413033,
         0.47917761, -1.05623228]])

In [16]:
y_pred_new_stock = model.predict(X_new_stock_scaled)
y_pred_new_stock

array([0, 0, 1, ..., 1, 0, 0])

In [17]:
print("Accuracy: ", accuracy_score(y_new_stock, y_pred_new_stock))

Accuracy:  0.9737698037981324


Create backtesting strategy

In [18]:
class trading_strategy(Strategy):
    n_train = 600
    coef_retrain = 200
    
    def init(self):
        self.model = XGBClassifier()
        self.model.fit(X=X_train, y=y_train)
        self.already_bought = False
        
    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :-1]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow == 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 0 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [19]:
"""class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each 200 days
        if len(self.data) % self.coef_retrain == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()"""

"class WalkForwardAnchored(Regression):\n    def next(self):\n        \n        # we don't take any action and move on to the following day\n        if len(self.data) < self.n_train:\n            return\n        \n        # we retrain the model each 200 days\n        if len(self.data) % self.coef_retrain == 0:\n            X_train = self.data.df.iloc[:, :-1]\n            y_train = self.data.df.iloc[:, -1]\n\n            self.model.fit(X_train, y_train)\n\n            super().next()\n            \n        else:\n            \n            super().next()"

In [20]:
bt = Backtest(df_new_stock, trading_strategy, cash=10000, commission=.002, exclusive_orders=True)

In [21]:
results = bt.run()
results

Start                     1986-04-18 00:00...
End                       2024-02-12 00:00...
Duration                  13814 days 00:00:00
Exposure Time [%]                   99.968524
Equity Final [$]          9108684568785971...
Equity Peak [$]           9126757868542632...
Return [%]                9108684568785972...
Buy & Hold Return [%]           408769.384601
Return (Ann.) [%]                 1186.499414
Volatility (Ann.) [%]              358.835685
Sharpe Ratio                         3.306526
Sortino Ratio                      107.071319
Calmar Ratio                        70.482975
Max. Drawdown [%]                  -16.833844
Avg. Drawdown [%]                   -1.391934
Max. Drawdown Duration       56 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                                 4842
Win Rate [%]                        79.429988
Best Trade [%]                      47.368692
Worst Trade [%]                    -14.638132
Avg. Trade [%]                    

Trade using Alpaca

In [55]:
#Read API keys from text file
with open('alpacaAPI.txt', 'r') as file:
    # Read the first line
    API_KEY, SECRET_KEY = file.read().split('\n')

In [60]:
import alpaca_trade_api as tradeapi

# Initialize the Alpaca API paper trading
api = tradeapi.REST(API_KEY,
                    SECRET_KEY,
                    base_url='https://paper-api.alpaca.markets')

# Submit a market order to buy 10 shares of AAPL
api.submit_order(
    symbol='ETH/USD',
    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc'
)

Order({   'asset_class': 'crypto',
    'asset_id': 'a1733398-6acc-4e92-af24-0d0667f78713',
    'canceled_at': None,
    'client_order_id': '2654b6f7-c27e-4fd8-bba3-f10a5ea79868',
    'created_at': '2024-02-13T23:58:58.629803603Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'c286dfde-552a-46b3-8562-0b7bdfd7a125',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2024-02-13T23:58:58.628368803Z',
    'subtag': None,
    'symbol': 'ETH/USD',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2024-02-13T23:58:58.629844223Z'})